In [1]:
from os.path import expanduser
import pandas as pd
import simplejson
import numpy as np
import urllib

In [2]:
# get the google api key
home = expanduser("~")
with open(home + '/.googleapi') as infile:
    apikey = infile.next().strip()

In [6]:
# read in the metadata file
meta_hdf_path = '/Users/bohr/workspace/capstone/DSE_Cohort2_Traffic_Capstone/C2 Exploratory/meta_2015.hdf'
meta_frame = pd.read_hdf(meta_hdf_path, 'meta_2015')
meta_frame.head()

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
584,1113072,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,0.452,ML,2,Black Mountain Rd,314,NaN,NaN,NaN,2015_01_01
585,1113073,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,OR,3,BLACK MOUNTAIN RD,314,NaN,NaN,NaN,2015_01_01
665,1113680,56,E,11,73,66000.0,6.862,7.364,32.953394,-117.133404,0.999,ML,3,BLACK MOUNTAIN RD,434,NaN,NaN,NaN,2015_01_01
666,1113683,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,FR,2,BLK MOUNTAIN - WB 56,314,NaN,NaN,NaN,2015_01_01
1034,1119041,125,N,11,73,NaN,1.433,3.282,32.608637,-116.967247,0.792,ML,3,1 MI S/O BIRCH RD,19106,NaN,NaN,NaN,2015_01_01


In [81]:
meta_frame.count()

ID           1541
Fwy          1541
Dir          1541
District     1541
County       1541
City         1318
State_PM     1541
Abs_PM       1541
Latitude     1539
Longitude    1539
Length        932
Type         1541
Lanes        1541
Name         1541
User_ID_1    1541
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1541
dtype: int64

In [103]:
meta_frame[meta_frame.Latitude.isnull()]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
767,1114649,805,S,11,73,66000.0,28.811,28.662,NaN,NaN,0.651,ML,4,S/B AT JCT I-5,4045,NaN,NaN,NaN,2015_12_17
1324,1125383,52,W,11,73,70224.0,14.756,14.756,NaN,NaN,0.695,ML,2,52 WB from 125 Conn,43812,NaN,NaN,NaN,2015_12_17


In [215]:
meta_frame_no_nan = meta_frame.dropna(axis='index', subset=['Latitude', 'Longitude'])
meta_frame_no_nan.index = np.arange(0, len(meta_frame_no_nan))
meta_frame_no_nan['lat_lon'] = meta_frame_no_nan['Latitude'].map(str) + ',' + meta_frame_no_nan['Longitude'].map(str)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [216]:
# spot check
meta_frame_no_nan[meta_frame_no_nan.ID == 1114649]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date,lat_lon


In [217]:
meta_frame_no_nan.count()

ID           1539
Fwy          1539
Dir          1539
District     1539
County       1539
City         1316
State_PM     1539
Abs_PM       1539
Latitude     1539
Longitude    1539
Length        930
Type         1539
Lanes        1539
Name         1539
User_ID_1    1539
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1539
lat_lon      1539
dtype: int64

In [118]:
ELEVATION_BASE_URL = 'https://maps.googleapis.com/maps/api/elevation/json'
def getElevation(lat_lon_list=None, **elvtn_args):
    """
    This function will get the elevation from the google api
    """
    result = None
    elvtn_args.update({
        'locations': "|".join(lat_lon_list),
        'key': apikey
    })

    url = ELEVATION_BASE_URL + '?' + urllib.urlencode(elvtn_args)
    response = simplejson.load(urllib.urlopen(url))

    if 'status' not in response or response['status'] != 'OK':
        print "bad response"
    else:
        result = [item['elevation'] for item in response['results']]
    return result

In [236]:
# # chunk the data into parts of 50
# id_master = []
# elevation_master = []

# chunk = 50
# for item in range(0, 31):
#     start = item*chunk
#     end = (item+1)*chunk
    
#     id_master += list(meta_frame_no_nan['ID'][start:end].values)
#     elevation_master += getElevation(list(meta_frame_no_nan['lat_lon'][start:end]))
    
# print len(id_master)
# print len(elevation_master)

1539
1539


In [237]:
elev_df = pd.DataFrame({'ID': id_master, 'elevation': elevation_master})

In [241]:
meta_frame.columns

Index([u'ID', u'Fwy', u'Dir', u'District', u'County', u'City', u'State_PM',
       u'Abs_PM', u'Latitude', u'Longitude', u'Length', u'Type', u'Lanes',
       u'Name', u'User_ID_1', u'User_ID_2', u'User_ID_3', u'User_ID_4',
       u'file_date'],
      dtype='object')

In [242]:
elev_df.columns

Index([u'ID', u'elevation'], dtype='object')

In [251]:
new_meta_frame = pd.merge(meta_frame, elev_df, how='left')

In [79]:
# find bad data...
new_meta_frame[new_meta_frame.elevation.isnull()]

,Unnamed: 0,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,...,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date,elevation
884,884,1114649,805,S,11,73,66000.0,28.811,28.662,NaN,...,0.651,ML,4,S/B AT JCT I-5,4045,NaN,NaN,NaN,2015_12_17,NaN
1441,1441,1125383,52,W,11,73,70224.0,14.756,14.756,NaN,...,0.695,ML,2,52 WB from 125 Conn,43812,NaN,NaN,NaN,2015_12_17,NaN


In [83]:
# Fix bad data via manual lookup
new_meta_frame.loc[884, 'Latitude'] = 32.966531
new_meta_frame.loc[884, 'Longitude'] = -117.2255
new_meta_frame.iloc[884]

Unnamed: 0               884
ID                   1114649
Fwy                      805
Dir                        S
District                  11
County                    73
City                   66000
State_PM              28.811
Abs_PM                28.662
Latitude             32.9665
Longitude           -117.225
Length                 0.651
Type                      ML
Lanes                      4
Name          S/B AT JCT I-5
User_ID_1               4045
User_ID_2                NaN
User_ID_3                NaN
User_ID_4                NaN
file_date         2015_12_17
elevation                NaN
Name: 884, dtype: object

In [85]:
new_meta_frame.loc[1441, 'Latitude'] = 32.836534
new_meta_frame.loc[1441, 'Longitude'] = -117.00755
new_meta_frame.iloc[1441]

Unnamed: 0                   1441
ID                        1125383
Fwy                            52
Dir                             W
District                       11
County                         73
City                        70224
State_PM                   14.756
Abs_PM                     14.756
Latitude                  32.8365
Longitude                -117.008
Length                      0.695
Type                           ML
Lanes                           2
Name          52 WB from 125 Conn
User_ID_1                   43812
User_ID_2                     NaN
User_ID_3                     NaN
User_ID_4                     NaN
file_date              2015_12_17
elevation                     NaN
Name: 1441, dtype: object

In [121]:
result_884 = getElevation(['32.9665,-117.2255'])
result_1441 = getElevation(['32.8365,-117.00755'])

In [123]:
new_meta_frame.loc[884, 'elevation'] = result_884[0]
new_meta_frame.loc[1441, 'elevation'] = result_1441[0]

In [124]:
# Note: all elevations are in meters.
new_meta_frame.to_csv('../data/meta_2015_with_elev.csv')

# Calculate slope between stations

In [125]:
new_meta_frame = pd.read_csv('../data/meta_2015_with_elev.csv')

In [126]:
def create_freeway_vectors():
    to_loop = new_meta_frame.groupby(['Fwy', 'Dir'])['ID'].count().reset_index()[['Fwy', 'Dir']].values
    
    ret = {}
    for Fwy, Dir in to_loop:
        if Dir == "N":
            sort_order = ('Abs_PM', True)        
        elif Dir == "S":
            sort_order = ('Abs_PM', False)        
        elif Dir == "E":
            sort_order = ('Abs_PM', True)        
        elif Dir == "W":
            sort_order = ('Abs_PM', False)
            
        ret["%s_%s" % (Fwy, Dir)] = new_meta_frame[(new_meta_frame.Fwy == Fwy) & (new_meta_frame.Dir == Dir)
                                                  & (new_meta_frame.Type == 'ML')]\
            .sort_values(by=sort_order[0], ascending=sort_order[1])[['ID', 'Latitude', 'Longitude', 'Abs_PM']]
    return ret 

In [127]:
freeway_vectors = create_freeway_vectors()

In [113]:
ELEVATION_BASE_URL = 'https://maps.googleapis.com/maps/api/elevation/json'
def getElevationPath(lat_lon_list=None, samples=10):
    """
    This function will get the elevation from the google api
    """
    result = None
    elvtn_args = {
        'path': "|".join(lat_lon_list),
        'samples': samples,
        'key': apikey
    }

    url = ELEVATION_BASE_URL + '?' + urllib.urlencode(elvtn_args)
    response = simplejson.load(urllib.urlopen(url))

    if 'status' not in response or response['status'] != 'OK':
        print "bad response"
        print "response: %s" % response
    else:
        result = [item['elevation'] for item in response['results']]
    return result

In [41]:
# Test the function
# result = getElevationPath(['32.834748,-117.003404', '32.809436,-117.005295'], samples=18)

In [116]:
# loop through all freeways and get the results
# calculations will be done afterwards to debug while not pulling from google's api
raw_results = {}
counter = 0
for key in freeway_vectors:
    print "on freeway: %s" % key
    if counter % 3:
        print "counter: %s" % counter
    frame = freeway_vectors[key]
    count = frame.ID.count()
    for index in range(1, count):
        distance = abs(frame.iloc[index-1].Abs_PM - frame.iloc[index].Abs_PM)

        # get enough samples for .1 resolution
        samples = int(round(distance, 1)*10)
        lat1, lon1 = frame.iloc[index-1][['Latitude', 'Longitude']]
        lat_lon1 = '%s,%s' % (lat1, lon1)
        lat2, lon2 = frame.iloc[index][['Latitude', 'Longitude']]
        lat_lon2 = '%s,%s' % (lat2, lon2)
        if samples < 2:
            samples = 2
        result = getElevationPath([lat_lon1, lat_lon2], samples=samples)
        
        start_id = frame.iloc[index-1].ID
        end_id = frame.iloc[index].ID
        if key not in raw_results:
            raw_results[key] = {}
        raw_results[key]['%s_%s' % (start_id, end_id)] = result
    counter += 1

on freeway: 125_S
on freeway: 905_E
on freeway: 1
on freeway: 54_W
on freeway: 2
on freeway: 125_N
on freeway: 52_W
on freeway: 4
on freeway: 67_N
on freeway: 5
on freeway: 56_E
on freeway: 67_S
on freeway: 7
on freeway: 56_W
on freeway: 8
on freeway: 905_W
on freeway: 94_E
on freeway: 10
on freeway: 52_E
on freeway: 11
on freeway: 78_W
on freeway: 5_N
on freeway: 13
on freeway: 15_N
on freeway: 14
on freeway: 15_S
on freeway: 94_W
on freeway: 16
on freeway: 78_E
on freeway: 17
on freeway: 54_E
on freeway: 5_S
on freeway: 19
on freeway: 8_E
on freeway: 20
on freeway: 163_N
on freeway: 805_S
on freeway: 22
on freeway: 8_W
on freeway: 23
on freeway: 805_N
on freeway: 163_S
on freeway: 25


In [133]:
# store results as a dataframe to csv just in case
# it's not pretty but I didn't design the data structure with a dataframe in mind...
df = pd.DataFrame(raw_results)
df.to_csv('raw_results.csv')

In [135]:
!ls -hl raw_results.csv

-rw-r--r--  1 bohr  staff   133K Mar  5 16:17 raw_results.csv


In [153]:
# calculate the grade for each station
id_list = []
grade_list = []
for key in freeway_vectors:
    frame = freeway_vectors[key]
    count = frame.ID.count()
    for index in range(1, count):
        # absolute value of run in case we walk the freeway backwards
        run = abs(frame.iloc[index-1].Abs_PM - frame.iloc[index].Abs_PM)
        # sometimes there are multiple stations on the same mile marker...
        if run == 0:
            run = abs(frame.iloc[index-2].Abs_PM - frame.iloc[index].Abs_PM)
#         print "run: %s" % run

        start_id = frame.iloc[index-1].ID
        end_id = frame.iloc[index].ID
        raw_elevations = raw_results[key]['%s_%s' % (start_id, end_id)]
#         print "raw_elevations: %s" % raw_elevations
        
        run_between_points = run/len(raw_elevations)
#         print "run_between_points: %s" % run_between_points

        # use the closest point to calculate the grade
        if index == count:
            # if at the end of the vector, then use the last raw_elevations
            grade = np.arctan((raw_elevations[-1] - raw_elevations[-2])/float(run_between_points))
        else:
            grade = np.arctan((raw_elevations[1] - raw_elevations[0])/float(run_between_points))
        
        id_list.append(start_id)
        grade_list.append(grade)

In [154]:
len(id_list)

815

In [155]:
len(grade_list)

815

In [159]:
grade_df = pd.DataFrame({'ID': id_list, 'grade': grade_list})

In [164]:
new_meta_frame_with_grade = pd.merge(new_meta_frame, grade_df, how='left', on='ID')

In [171]:
new_meta_frame_with_grade[new_meta_frame_with_grade.Type == 'ML'].to_csv('../data/meta_2015_with_elev_and_grade.csv')